# HW 2 Multimodal Machine Learning for Emotion Recognition

- main (this notebook) with sub notebooks
    1. audio (acoustic)
    2. text (lexical)
    3. visual
- IEMOCAP (Interactive Emotional Dyadic Motion Capture) database

# TODOs
In the file dataset.csv, you are provided with the relative address for the audio, visual and text feature files alongwith their corresponding emotion labels. There are 5 sessions and each session has one male and one female speaker.

1. You can use different pooling methods (e.g., max pooling, mean pooling) for reducing the temporal dimensionof the audio and visual files, or use your preferred temporal modeling (e.g., RNN, GRU, LSTM) to obtainfeature vectors per data point.1

2. Perform a 4-class emotion classification using your preferred classifier with the obtained feature vectors. Selectthe parameters using Grid Search (search over a range for hyper-parameters). Perform any additional stepsyou see fit to obtain the best results.

3. Report your classification results on individual modalities (vision, speech, and text) using F1-micro metricon a 10-fold subject-independent cross validation.

4. How do you handle the problem of class imbalance? Plot the confusion matrix for the 4 classes.

5. Use both early fusion (concatenate features from different modalities) and late fusion (majority vote over theoutputs of the unimodal models) to obtain multimodal classification results. Report and compare the resultsfor both fusion techniques.

6. Provide an interpretation on your results from the performed unimodal and multimodal classification tasks.Which one is performing best and why?

*Note*: You are only allowed to use the features and labels provided by us with this assignment. Please refrainfrom using the original data; assignments submitted with any other labels or data will not be graded

#  Imports + Load Data

In [1]:
import pandas as pd
import numpy as np
import librosa, librosa.display
import matplotlib.pyplot as plt

In [65]:
BASE = "/Users/brinkley97/Documents/development/"
CLASS_PATH = "classes/csci_535_multimodal_probabilistic_learning/"
DATASET_PATH = "datasets/hw_2"

SESSION_1 = "Session1/"
SESSION_2 = "Session2"
SESSION_3 = "Session3/"
SESSION_4 = "Session4/"
SESSION_5 = "Session5/"

SES_01F = "Ses01F_impro01/"

FILE = "/iemocapRelativeAddressForFiles.csv"
file_paths = BASE + CLASS_PATH + DATASET_PATH + FILE

In [66]:
def load_data(file):
    original_data = pd.read_csv(file)
    # original_data = pd.DataFrame(file)
    copy_of_data = original_data.copy()
    return copy_of_data

In [67]:
# 4 classes - anger(0), sadness(1) and happiness(2),and neutral(3)
dataset_paths_copy = load_data(file_paths)
dataset_paths_copy

file_name_list speakers  \
0        Ses01F_impro01_F001      F01   
1        Ses01F_impro01_M011      M01   
2        Ses01F_impro02_F002      F01   
3        Ses01F_impro02_F003      F01   
4        Ses01F_impro02_F004      F01   
...                      ...      ...   
1331  Ses05M_script03_2_M029      M05   
1332  Ses05M_script03_2_M039      M05   
1333  Ses05M_script03_2_M041      M05   
1334  Ses05M_script03_2_M042      M05   
1335  Ses05M_script03_2_M043      M05   

                                                                      visual_features  \
0           /features/visual_features/Session1/Ses01F_impro01/Ses01F_impro01_F001.npy   
1           /features/visual_features/Session1/Ses01F_impro01/Ses01F_impro01_M011.npy   
2           /features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F002.npy   
3           /features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F003.npy   
4           /features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F004.npy   
...                                                                               ...   
1331  /features/visual_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M029.npy   
1332  /features/visual_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M039.npy   
1333  /features/visual_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M041.npy   
1334  /features/visual_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M042.npy   
1335  /features/visual_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M043.npy   

                                                                      acoustic_features  \
0           /features/acoustic_features/Session1/Ses01F_impro01/Ses01F_impro01_F001.npy   
1           /features/acoustic_features/Session1/Ses01F_impro01/Ses01F_impro01_M011.npy   
2           /features/acoustic_features/Session1/Ses01F_impro02/Ses01F_impro02_F002.npy   
3           /features/acoustic_features/Session1/Ses01F_impro02/Ses01F_impro02_F003.npy   
4           /features/acoustic_features/Session1/Ses01F_impro02/Ses01F_impro02_F004.npy   
...                                                                                 ...   
1331  /features/acoustic_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M029.npy   
1332  /features/acoustic_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M039.npy   
1333  /features/acoustic_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M041.npy   
1334  /features/acoustic_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M042.npy   
1335  /features/acoustic_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M043.npy   

                                                                      lexical_features  \
0           /features/lexical_features/Session1/Ses01F_impro01/Ses01F_impro01_F001.npy   
1           /features/lexical_features/Session1/Ses01F_impro01/Ses01F_impro01_M011.npy   
2           /features/lexical_features/Session1/Ses01F_impro02/Ses01F_impro02_F002.npy   
3           /features/lexical_features/Session1/Ses01F_impro02/Ses01F_impro02_F003.npy   
4           /features/lexical_features/Session1/Ses01F_impro02/Ses01F_impro02_F004.npy   
...                                                                                ...   
1331  /features/lexical_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M029.npy   
1332  /features/lexical_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M039.npy   
1333  /features/lexical_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M041.npy   
1334  /features/lexical_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M042.npy   
1335  /features/lexical_features/Session5/Ses05M_script03_2/Ses05M_script03_2_M043.npy   

      emotion_labels  
0                  3  
1                  0  
2                  1  
3                  3  
4                  1  
...              ...  
1331               0  
1332               0  
1333               0  
1334               0  
1335               0  

[

# Preprocessing Files

- [x] Build paths to specific files
- [ ] Reduce the time (temporal) dimension

In [95]:
def build_paths_to_file(df_with_paths, specific_feature):
    """With the given DataFrame of paths, build my paths...
    
    Parameters
    df_with_paths -- 
    specific_feature -- str (either visual_features, acoustic_features, or lexical_features)
    
    Return
    list
    """
    
    features = df_with_paths.loc[0:, ['file_name_list', 'speakers', specific_feature]]
    features_path = features.loc[0:, specific_feature]
    features["file_with_path"] = BASE + CLASS_PATH + DATASET_PATH + features_path
    list_of_features = list(features["file_with_path"])
    
    return list_of_features

In [ ]:
specific_feature = 'acoustic_features'
audio_features = build_paths_to_file(dataset_paths_copy, specific_feature)

specific_feature = 'lexical_features'
text_features = build_paths_to_file(dataset_paths_copy, specific_feature)


specific_feature = 'visual_features'
visual_features = build_paths_to_file(dataset_paths_copy, specific_feature)

## Load Audio (Acoustic) features
- We have VGGish, a deep convolutional neural network pre-trained on audio spectrograms extracted from a large database of videos to recognize a large variety of audio event categories [3]. The 128-dimensional embeddings were generated by VGGish after dimensionality reduction with Principal Component Analysis (PCA).

In [96]:
audio_features = BASE + CLASS_PATH + DATASET_PATH + '/features/acoustic_features/' + SESSION_1 + SES_01F
audio_features
female_s1 = audio_features + 'Ses01F_impro01_F001.npy'
male_s1 = audio_features + 'Ses01F_impro01_M011.npy'

In [97]:
female_s1_load = np.load(female_s1)
male_s1_load = np.load(male_s1)

In [103]:
# audio_features

## Load Text (Lexical) features

In [98]:
pd.set_option('max_colwidth', 200)

In [99]:
# text_features

## Load Visual features

In [101]:
# visual_features

['/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro01/Ses01F_impro01_F001.npy',
 '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro01/Ses01F_impro01_M011.npy',
 '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F002.npy',
 '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F003.npy',
 '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learning/datasets/hw_2/features/visual_features/Session1/Ses01F_impro02/Ses01F_impro02_F004.npy',
 '/Users/brinkley97/Documents/development/classes/csci_535_multimodal_probabilistic_learni